In [1]:
# Importing dependencies
import os
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# Path to source JSON
businessJson=os.path.join('sourceData', 'business.json')

In [3]:
# Path to Yelp food/restaurant categories csv
yelpCategories=os.path.join('sourceData', 'yelpCategories.csv')

In [4]:
# Creating pd dataframe
business_raw=pd.read_json(businessJson, lines=True)

In [5]:
# Select only the businesses in Ontario
business_on=business_raw.loc[business_raw['state'] == 'ON']

In [6]:
# Dropping any rows with blank values in these categories
business=business_on.dropna(subset=['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude', 'attributes',
                                               'categories', 'hours']).reset_index(drop=True)

In [7]:
# Regex to fix spelling mistakes 
business.replace({'city': {'^AGINCOURT$': 'Agincourt',
                           '^Bradford West Gwillimbury$': 'Bradford',
                           '^East Ajax$': 'Ajax',
                           '^Caledon.{,8}$': 'Caledon',
                           '^East Gwil{1,2}imbury$': 'East Gwillimbury',
                           '(?i)^.*icoke$': 'Etobicoke',
                           '^.{,9}Toro?nto.{,9}$': 'Toronto',
                           'Malton': 'Mississauga',
                           '^.{,5}Missis{1,2}a?ua?g.{1,2}$': 'Mississauga',
                           '^Regional Municipality of York$': 'North York',
                           '(?i)^North.{0,2}York$': 'North York',
                           '^York Regional Municipality$': 'York',
                           '^Willowdale$': 'North York',
                           '^North of Brampton$': 'Brampton',
                           '(?i)^Oak.?ridges$': 'Oak Ridges',
                           '^oakville$': 'Oakville',
                           '(?i)^Richmond?.?Hill?$': 'Richmond Hill',
                           '^.{,8}Scar.?bo?rough$': 'Scarborough',
                           '^.{,11}Stouffville$': 'Stouffville',
                           '(?i)^Thornhil{,2}$': 'Thornhill',
                           '^.*Vaugh.{,3}$': 'Vaughan',
                           '^Wh.?i.?by$': 'Whitby'}}, inplace=True, regex=True)

In [8]:
# Only taking these columns
business=business.loc[:, ['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude','categories', 'stars', 'hours', 'attributes']]
business.columns=['Name', 'Address', 'Postal_code', 'City', 'Province', 'Latitude', 'Longitude', 'Categories', 'Stars', 'Hours', 'Attributes']

In [9]:
# Turning the hours column in to a df
hours_raw=json_normalize(data=business['Hours'])
business.drop(columns='Hours', inplace=True)
# hours_raw.head()

## Handling the hours column

In [10]:
# Reorganise columns
hours_raw=hours_raw.loc[:,['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [11]:
# Create a new df with opening and closing hours
columnsHours=hours_raw.columns
hours=hours_raw

In [12]:
# Loop through and split the columns
for column in columnsHours:
    hours[[f"{column}_open", f"{column}_close"]]=hours_raw[column].str.split('-', expand=True)
hours.drop(columns=columnsHours, inplace=True)
hours=hours.apply(lambda x: x.str.strip())

In [13]:
# Add hours column to the original DF
business=business.join(hours)

## Handling the categories column

In [14]:
# Turning the Categories column in to a df and adding restaurant id
category = business['Categories'].str.split(',', expand=True)
business.drop(columns='Categories', inplace=True)
category['Restaurant_id']=business.index
category=pd.melt(category, id_vars='Restaurant_id', value_name='Category').drop(columns='variable').sort_values('Restaurant_id')

In [15]:
# Eliminating None categories
category=category.loc[~category['Category'].isin([None])].reset_index(drop=True)
category['Category'] = category['Category'].str.strip()

In [16]:
# Read csv with list of yelp categories
foodCategories=pd.read_csv(yelpCategories)
foodCategories['Category'] = foodCategories['Category'].str.strip()

In [17]:
# Take only the categories from the list of yelp categories
categoryFiltered=pd.DataFrame(category.loc[category['Category'].isin(foodCategories['Category'])])

In [18]:
# Filter out categories that don't apply
categoryFiltered=categoryFiltered.loc[category['Category'].isin(['Acai Bowls', 'Afghan', 'African', 'Arabian', 'Argentine', 'Asian Fusion', 'Australian', 'Austrian', 'Bagels', 'Bangladeshi',
                                                                   'Barbeque', 'Basque', 'Beer Bar', 'Beer Gardens', 'Beer Hall', 'Belgian', 'Bistros', 'Brasseries', 'Brazilian', 'Breakfast & Brunch',
                                                                   'Brewpubs', 'British', 'Bubble Tea', 'Buffets', 'Burgers', 'Burmese', 'Cafes', 'Cajun/Creole', 'Cambodian', 'Cantonese', 'Caribbean',
                                                                   'Champagne Bars', 'Cheese Shops', 'Cheesesteaks', 'Chicken Shop', 'Chicken Wings', 'Chinese', 'Chocolatiers & Shops',
                                                                   'Cocktail Bars', 'Coffee & Tea', 'Colombian', 'Comfort Food', 'Creperies', 'Cuban', 'Cupcakes',
                                                                   'Czech', 'Czech/Slovakian', 'Delicatessen', 'Delis', 'Desserts', 'Dim Sum', 'Diners', 'Dive Bars', 'Do-It-Yourself Food', 'Donairs',
                                                                   'Egyptian', 'Ethical Grocery', 'Ethiopian', 'Falafel', 'Fast Food', 'Filipino', 'Fish & Chips', 'Fondue',
                                                                   'Food Stands', 'Food Trucks', 'French', 'Fruits & Veggies', 'Gastropubs', 'Gelato', 'German', 'Gluten-Free', 'Greek', 'Hainan', 'Haitian',
                                                                   'Hakka', 'Halal', 'Hawaiian', 'Himalayan/Nepalese', 'Hong Kong Style Cafe', 'Hot Dogs', 'Hot Pot', 'Hungarian', 'Iberian',
                                                                   'Ice Cream & Frozen Yogurt', 'Imported Food', 'Indian', 'Indonesian', 'International', 'Internet Cafes', 'Irish', 'Irish Pub', 'Italian',
                                                                   'Izakaya', 'Japanese', 'Japanese Sweets', 'Juice Bars & Smoothies', 'Kebab', 'Kombucha', 'Korean', 'Kosher', 'Laotian', 'Latin American',
                                                                   'Lebanese', 'Live/Raw Food', 'Lounges', 'Macarons', 'Malaysian', 'Mauritius', 'Mediterranean', 'Mexican', 'Middle Eastern',
                                                                   'Milkshake Bars', 'Minho', 'Modern European', 'Mongolian', 'Moroccan', 'Nicaraguan', 'Noodles', 'Pakistani', 'Pan Asian', 'Pasta Shops',
                                                                   'Persian/Iranian', 'Peruvian', 'Pizza', 'Poke', 'Polish', 'Portuguese',
                                                                   'Poutineries', 'Pubs', 'Ramen', 'Reunion', 'Russian', 'Salad', 'Salvadoran', 'Sandwiches', 'Scandinavian', 'Scottish', 'Seafood',
                                                                   'Seafood Markets', 'Shanghainese', 'Shaved Ice', 'Shaved Snow', 'Singaporean', 'Slovakian', 'Smokehouse', 'Soul Food', 'Soup',
                                                                   'South African', 'Southern', 'Spanish', 'Specialty Food', 'Sports Bars', 'Sri Lankan', 'Steakhouses', 'Street Vendors', 'Supper Clubs',
                                                                   'Sushi Bars', 'Swiss Food', 'Syrian', 'Szechuan', 'Tacos', 'Taiwanese', 'Tapas Bars', 'Tapas/Small Plates', 'Tea Rooms', 'Tempura',
                                                                   'Teppanyaki', 'Tex-Mex', 'Thai', 'Themed Cafes', 'Tiki Bars', 'Tonkatsu', 'Trinidadian', 'Turkish', 'Udon', 'Ukrainian', 'Vegan',
                                                                   'Vegetarian', 'Venezuelan', 'Vietnamese', 'Waffles', 'Whiskey Bars', 'Wine Bars', 'Wraps'])]

In [19]:
# Creating a list of unique food categories
uniqueCategories=categoryFiltered['Category'].unique()
uniqueCategories.sort()

In [20]:
# Replace all the values in categoryFiltered with the uniqueCategories index
for index, value in enumerate(uniqueCategories):
    categoryFiltered['Category'].replace(value, str(index), inplace=True)

In [21]:
# Collect all the categories to one string
categoryPivot=categoryFiltered.pivot('Restaurant_id', 'Category', 'Category')
categoryPivot['Categories']=categoryPivot.apply(lambda x: ','.join(x.dropna().values), axis=1)

In [22]:
# Join categories column on business
business=business.join(categoryPivot['Categories'], how='inner').reset_index(drop=True)
business.rename(columns={'Categories':'Category_ids'}, inplace=True)

## Handling the attributes column

In [23]:
# Turning the Attributes column in to a df and adding restaurant id
attributesRaw=json_normalize(business['Attributes'])
attributesRaw['Restaurant_id']=business.index
business.drop(columns='Attributes', inplace=True)
attributes=attributesRaw.loc[attributesRaw['ByAppointmentOnly']!='True']

In [24]:
# Generate unique categories for ambience 
ambienceCategories = attributesRaw['Ambience'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
ambienceCategories=ambienceCategories.loc[~ambienceCategories['value'].isin(['None', 'NaN', None])].dropna()
ambienceCategories=pd.DataFrame(ambienceCategories['value'].unique())
ambienceCategories=ambienceCategories.sort_values(0).reset_index(drop=True)
ambienceCategories=ambienceCategories.replace("'", '', regex=True)
ambienceCategories=ambienceCategories[0].tolist()

In [25]:
# Generating boolean columns
for value in ambienceCategories:
    attributes[f'Ambience_{value}']=attributesRaw['Ambience'].str.contains(f"{value}': True")
attributes.drop(columns='Ambience', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [26]:
# Generate unique categories for BusinessParking 
businessParking = attributesRaw['BusinessParking'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
businessParking=businessParking.loc[~businessParking['value'].isin(['None', 'NaN', None])].dropna()
businessParking=pd.DataFrame(businessParking['value'].unique())
businessParking=businessParking.sort_values(0).reset_index(drop=True)
businessParking=businessParking.replace(["'", ' '], '', regex=True)
businessParking.drop(index=0, inplace=True)
businessParking=businessParking[0].unique().tolist()

In [27]:
# Generating boolean columns 
for value in businessParking:
    attributes[f'Parking_{value}']=attributesRaw['BusinessParking'].str.contains(f"{value}': True")
attributes.drop(columns='BusinessParking', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [28]:
# Generate unique categories for GoodForMeal 
goodForMeal = attributesRaw['GoodForMeal'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
goodForMeal=goodForMeal.loc[~goodForMeal['value'].isin(['None', 'NaN', None])].dropna()
goodForMeal=pd.DataFrame(goodForMeal['value'].unique())
goodForMeal=goodForMeal.sort_values(0).reset_index(drop=True)
goodForMeal=goodForMeal.replace(["'", ' '], '', regex=True)
goodForMeal.drop(index=0, inplace=True)
goodForMeal=goodForMeal[0].unique().tolist()

In [29]:
# Generating boolean columns
for value in goodForMeal:
    attributes[f'Meal_{value}']=attributesRaw['GoodForMeal'].str.contains(f"{value}': True")
attributes.drop(columns='GoodForMeal', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
# Generate unique categories for BestNights 
bestNights = attributesRaw['BestNights'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
bestNights=bestNights.loc[~bestNights['value'].isin(['None', 'NaN', None])].dropna()
bestNights=pd.DataFrame(bestNights['value'].unique())
bestNights=bestNights.sort_values(0).reset_index(drop=True)
bestNights=bestNights.replace(["'", ' '], '', regex=True)
bestNights.drop(index=0, inplace=True)
bestNights=bestNights[0].unique().tolist()

In [31]:
# Generating boolean columns
for value in bestNights:
    attributes[f'Best_night_{value}']=attributesRaw['BestNights'].str.contains(f"{value}': True")
attributes.drop(columns='BestNights', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [32]:
# Generate unique categories for Alcohol 
alcohol=pd.DataFrame(attributesRaw['Alcohol'].unique()).dropna()
alcohol=alcohol.replace(["'", ' '], '', regex=True)
alcohol.drop(index=[0, 2, 3, 5, 7], inplace=True)
alcohol=alcohol[0].tolist()

In [33]:
# Generate unique categories for Alcohol 
for value in alcohol:
    attributes[f'Alcohol_{value}']=attributesRaw['Alcohol'].str.contains(value)
attributes.drop(columns='Alcohol', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
# Generate unique categories for DietaryRestrictions 
dietaryRestrictions = attributesRaw['DietaryRestrictions'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
dietaryRestrictions=dietaryRestrictions.loc[~dietaryRestrictions['value'].isin(['None', 'NaN', None])].dropna()
dietaryRestrictions=pd.DataFrame(dietaryRestrictions['value'].unique())
dietaryRestrictions=dietaryRestrictions.sort_values(0).reset_index(drop=True)
dietaryRestrictions=dietaryRestrictions.replace(["'", ' '], '', regex=True)
dietaryRestrictions.drop(index=0, inplace=True)
dietaryRestrictions=dietaryRestrictions[0].unique().tolist()

['halal', 'kosher', 'soy-free', 'vegan', 'vegetarian', 'dairy-free']

In [37]:
# Generating boolean columns
for value in dietaryRestrictions:
    attributes[f'Dietary_Restrictions_{value}']=attributesRaw['DietaryRestrictions'].str.contains(f"{value}': True")
attributes.drop(columns='DietaryRestrictions', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [54]:
# Generate unique categories for Music 
music = attributesRaw['Music'].str.split(',', expand=True).replace(['{', '}', 'False', 'True', ':', "'"], '', regex=True).melt().drop(columns='variable')
music=music.loc[~music['value'].isin(['None', 'NaN', None])].dropna()
music=pd.DataFrame(music['value'].unique())
music=music.sort_values(0).reset_index(drop=True)
music=music.replace(["'", ' '], '', regex=True)
music.drop(index=0, inplace=True)
music=music[0].unique().tolist()

['background_music', 'dj', 'jukebox', 'karaoke', 'live', 'no_music', 'video']

In [55]:
# Generating boolean columns
for value in music:
    attributes[f'Music_{value}']=attributesRaw['Music'].str.contains(f"{value}': True")
attributes.drop(columns='Music', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [65]:
# Generate unique categories for NoiseLevel 
noiseLevel=pd.DataFrame(attributesRaw['NoiseLevel'].unique()).dropna().reset_index(drop=True)
noiseLevel=noiseLevel.replace(["'", ' '], '', regex=True)
noiseLevel.drop(index=[0, 1, 3, 6, 8], inplace=True)
noiseLevel=noiseLevel[0].tolist()
noiseLevel

['quiet', 'average', 'loud', 'very_loud']

In [70]:
# Generate unique categories for noiseLevel 
for value in noiseLevel:
    attributes[f'Noise_{value}']=attributesRaw['NoiseLevel'].str.contains(value)
attributes.drop(columns='NoiseLevel', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "['NoiseLevel'] not found in axis"

In [88]:
# Generate unique categories for RestaurantsAttire 
restaurantsAttire=pd.DataFrame(attributesRaw['RestaurantsAttire'].unique()).dropna().reset_index(drop=True)
restaurantsAttire=restaurantsAttire.replace(["'", ' '], '', regex=True).reset_index(drop=True)
restaurantsAttire.drop(index=[0, 3, 4, 5], inplace=True)
restaurantsAttire=restaurantsAttire[0].tolist()

,0
0,ucasual
1,dressy
2,casual
3,None
4,udressy
5,uformal
6,formal


In [78]:
# Generate unique categories for RestaurantsAttire 
for value in restaurantsAttire:
    attributes[f'Restaurants_Attire_{value}']=attributesRaw['RestaurantsAttire'].str.contains(value)
attributes.drop(columns='RestaurantsAttire', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Users\brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [109]:
# Generate unique categories for Smoking 
smoking=pd.DataFrame(attributesRaw['Smoking'].unique()).dropna().reset_index(drop=True)
smoking=smoking.replace(["'", "' '", '.outdoor', '.yes'], ['', '', 'outdoor', 'yes'], regex=True).reset_index(drop=True)
smoking.drop(index=[0, 2], inplace=True)
smoking=smoking[0].tolist()

In [110]:
# Generating boolean columns
for value in smoking:
    attributes[f'Smoking_{value}']=attributesRaw['Smoking'].str.contains(f"{value}': True")
attributes.drop(columns='Smoking', inplace=True)

E:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
attributes.drop(columns=['Caters', 'DriveThru', 'HairSpecializesIn', 'Open24Hours', 'RestaurantsDelivery'], inplace=True)

In [86]:
attributes.columns

Index(['AcceptsInsurance', 'AgesAllowed', 'BYOBCorkage', 'BikeParking',
       'BusinessAcceptsBitcoin', 'BusinessAcceptsCreditCards',
       'ByAppointmentOnly', 'Caters', 'CoatCheck', 'DogsAllowed', 'DriveThru',
       'GoodForDancing', 'GoodForKids', 'HairSpecializesIn', 'HappyHour',
       'HasTV', 'Open24Hours', 'OutdoorSeating', 'RestaurantsCounterService',
       'RestaurantsDelivery', 'RestaurantsGoodForGroups',
       'RestaurantsPriceRange2', 'RestaurantsReservations',
       'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking',
       'WheelchairAccessible', 'WiFi', 'Restaurant_id', 'Ambience_casual',
       'Ambience_classy', 'Ambience_divey', 'Ambience_hipster',
       'Ambience_intimate', 'Ambience_romantic', 'Ambience_touristy',
       'Ambience_trendy', 'Ambience_upscale', 'Parking_garage', 'Parking_lot',
       'Parking_street', 'Parking_valet', 'Parking_validated',
       'Meal_breakfast', 'Meal_brunch', 'Meal_dinner', 'Meal_latenight',
       'Meal_lunch', 'Mea

In [112]:
attributes['WiFi'].unique()

array(["u'no'", "u'free'", "'no'", nan, "'free'", "u'paid'", "'paid'",
       'None'], dtype=object)

# TODO check if all the boolean columns have values in them by running attributes['x'].sum()